#!pip install -U pandas-profiling


In [91]:
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in /Users/karthikmahendra/anaconda3/lib/python3.7/site-packages (2.1.2)


In [123]:
!pip install -U util

  Could not find a version that satisfies the requirement util (from versions: )
No matching distribution found for util


In [130]:
import numpy as np 
import pandas as pd
import pandas_profiling 
import datetime as dt
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 
import category_encoders as ce
from sklearn.model_selection import train_test_split


In [93]:
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train_features = pd.read_csv(WEB + 'train_features.csv')
train_labels = pd.read_csv(WEB + 'train_labels.csv')
test_features = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

PRE-PROCESS 

In [94]:
#Explore data

profile = train_features.profile_report()
profile.to_file(output_file = 'output.html')

In [95]:
train_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [96]:
# from output.html we have 10 numerical and 26 categorical features. 2 boolean and 2 rejected. 

bool_features = ['public_meeting','permit']
rejected_features = ['quantity_group','recorded_by']
categorical_features = train_features.select_dtypes(exclude=[np.number]).columns.to_list()
numerical_features = train_features.select_dtypes(include=[np.number]).columns.to_list()

In [97]:
#Remove boolean values from categorical_features
categorical_features = list(set(categorical_features) - set(bool_features))

In [98]:
print(categorical_features)

['scheme_management', 'payment', 'waterpoint_type', 'scheme_name', 'funder', 'source_class', 'extraction_type', 'management_group', 'water_quality', 'basin', 'recorded_by', 'source', 'extraction_type_class', 'ward', 'installer', 'date_recorded', 'extraction_type_group', 'lga', 'wpt_name', 'quantity', 'subvillage', 'management', 'quality_group', 'source_type', 'payment_type', 'quantity_group', 'waterpoint_type_group', 'region']


In [99]:
print(numerical_features)

['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']


In [100]:
# find if there are any NANs in categorical_features 

train_features[categorical_features].isnull().values.any()

True

In [101]:
# find if there are any NANs in numerical_features 

train_features[numerical_features].isnull().values.any()

False

In [102]:
#Since null values are only in categorical features, 
#I will find the top values in each and replace NANs with that value
def clean_data(lis):
    for element in lis:
        top = train_features[element].describe().to_list()[2]
        train_features[element]=train_features[element].fillna(top)
    

In [103]:
clean_data(categorical_features)

In [104]:
train_features[categorical_features].isnull().values.any()

False

In [105]:
train_features.isnull().values.any()

True

SPLIT

In [128]:
X = train_features
y = train_labels['status_group']
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

In [129]:
type(X_train),y_train.shape,X_val.shape,y_val.shape


(pandas.core.frame.DataFrame, (44550,), (14850, 40), (14850,))

LOGISTIC REGRESSION

In [117]:
# Select Categorical features with <19 values to train dataset
cat_5 = []
for ele in categorical_features:
    if len(train_features[ele].unique().tolist())<19:
        #print(f'{ele}: {len(train_features[ele].unique().tolist())}')
        cat_5.append(ele)
print (cat_5)
    

['scheme_management', 'payment', 'waterpoint_type', 'source_class', 'extraction_type', 'management_group', 'water_quality', 'basin', 'recorded_by', 'source', 'extraction_type_class', 'extraction_type_group', 'quantity', 'management', 'quality_group', 'source_type', 'payment_type', 'quantity_group', 'waterpoint_type_group']


In [118]:
features = cat_5 + numerical_features

In [119]:
X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset[cat_5])
X_val_encoded = encoder.transform(X_val_subset[cat_5])

scaler = StandardScaler()
X_train_subset[numerical_features] = scaler.fit_transform(X_train_subset[numerical_features])
X_val_subset[numerical_features] = scaler.transform(X_val_subset[numerical_features])



/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be

In [120]:
model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000)
model.fit(X_train_encoded, y_train)
print('Validation Accuracy', model.score(X_val_encoded, y_val))

Validation Accuracy 0.7291432846988403


PREDICTION

In [124]:
y_pred = model.predict(X_val)

ValueError: could not convert string to float: '2013-02-03'

In [127]:
X_test.shape,test_features.shape

((5940, 40), (14358, 40))

In [ ]:
# estimator is your scikit-learn estimator, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = estimator.predict(X_test)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)

Submission to Kaggle 